In [1]:
import datetime as dt
import difflib
import json
import math
import os
import string
import sys
from datetime import timedelta
from random import sample
import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pyreadstat
import seaborn as sns
import statsmodels.tools.tools as sm
import statsmodels.base as sb
from folium.features import CustomIcon
from folium.plugins import FastMarkerCluster, HeatMap, MarkerCluster
from linearmodels.panel.model import PooledOLS
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tools.eval_measures import aic, rmse
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.svar_model import SVAR
from statsmodels.tsa.stattools import adfuller
import yfinance as yf
from yahoofinancials import YahooFinancials
import pickle
import array_to_latex as a2l
import markdown
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import pdfkit as pdf
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from scipy.signal import savgol_filter

In [2]:
# The name of the original Data is changed by me. 
data = pd.read_csv(
    "./markettrends0.csv",
    dtype={
        "state": "str",
        "sa3_name16": "str",
        "sa4_name16": "str",
        "postcode": "str",
        "state": "str",
        "property_type": "str",
    },
)
ndata = data.fillna(
    {"Volume of new rental listings (1 month)": 0, "Volume of sales (1 month)": 0}
).dropna(subset=["postcode"])
ndatahouses = ndata[:][ndata.property_type == "Houses"]
ndataunits = ndata[:][ndata.property_type == "Units"]

In [3]:
with open('DataFrameDict_postcode.pickle', 'rb') as f:
     DataFrameDict_postcode =  pickle.load(f) 
        
aveHPI_lv3 = []
lst = ["postcode", "sa4", "state", "logHPI", "logHPIdiff"]
# Calling DataFrame constructor on list
for key in DataFrameDict_postcode.keys():
    df = pd.DataFrame([], columns=lst)
    df["Description"] = DataFrameDict_postcode[key]["value_at_date"].loc[
        ("2021-06" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")
    ]
    df["logHPI"] =np.log2(
        DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
            ("2021-06" > DataFrameDict_postcode[key]["value_at_date"])
            & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")
        ]
    )
    df["postcode"] = key
    df["sa4"] = DataFrameDict_postcode[key]["sa4_name16"].loc[
        ( DataFrameDict_postcode[key]["postcode"] == key)]
    df["state"] = DataFrameDict_postcode[key]["state"].loc[
        ( DataFrameDict_postcode[key]["postcode"] == key)]
    
    df["logHPIdiff"] = df["logHPI"].diff(1)
    aveHPI_lv3.append(df)

aveHPIdf_lv3 = pd.concat(aveHPI_lv3)
aveHPIdf_lv3 = aveHPIdf_lv3.reset_index(drop=True)
aveHPIdf_lv3['date'] = pd.to_datetime(aveHPIdf_lv3['Description'],errors = 'coerce')
aveHPIdf_lv3['year'] = pd.DatetimeIndex(aveHPIdf_lv3['date']).year

In [120]:
aveHPIdf_lv3 = aveHPIdf_lv3[aveHPIdf_lv3['state'] == 'NSW']
aveHPIdf_lv3 = aveHPIdf_lv3[aveHPIdf_lv3['date'] > '2009-12-31']

In [121]:
aveHPIdf_lv3 

,postcode,sa4,state,logHPI,logHPIdiff,Description,date,year
121,2018,Sydney - City and Inner South,NSW,6.657473,0.013617,2010-01-31,2010-01-31,2010
122,2018,Sydney - City and Inner South,NSW,6.676620,0.019147,2010-02-28,2010-02-28,2010
123,2018,Sydney - City and Inner South,NSW,6.707935,0.031315,2010-03-31,2010-03-31,2010
124,2018,Sydney - City and Inner South,NSW,6.718974,0.011039,2010-04-30,2010-04-30,2010
125,2018,Sydney - City and Inner South,NSW,6.730457,0.011483,2010-05-31,2010-05-31,2010
...,...,...,...,...,...,...,...,...
36889,2737,Murray,NSW,7.612153,0.019043,2021-01-31,2021-01-31,2021
36890,2737,Murray,NSW,7.714129,0.101976,2021-02-28,2021-02-28,2021
36891,2737,Murray,NSW,7.715083,0.000954,2021-03-31,2021-03-31,2021
36892,2737,Murray,NSW,7.735679,0.020595,2021-04-30,2021-04-30,2021


In [677]:
#NSW(The scope of data is narrowed down to the NSW from the national level)
avg_1 = aveHPIdf_lv3.groupby(["date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_1 = avg_1.rename(columns = {'mean':'l0'}).dropna().reset_index(drop = True)

sum_1 = aveHPIdf_lv3.groupby(["date"])["logHPIdiff"].agg(["sum"]).reset_index()
sum_1 = sum_1.rename(columns = {'sum':'l0_s'}).dropna().reset_index(drop = True)
sum_1 = sum_1.reset_index(drop=True)

#sa4 
avg_2 = aveHPIdf_lv3.groupby(["sa4","date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_2 = avg_2.rename(columns = {'mean':'l01'}).dropna().reset_index(drop = True)

sum_2 = aveHPIdf_lv3.groupby(["sa4","date"])["logHPIdiff"].agg(["sum"]).reset_index()
sum_2 = sum_2.rename(columns = {'sum':'l01_s'}).dropna().reset_index(drop = True)
sum_2 = sum_2.reset_index(drop=True)

#postcode
avg_3 = aveHPIdf_lv3.groupby(["postcode","date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_3 = avg_3.rename(columns = {'mean':'l012'}).dropna().reset_index(drop = True)


In [125]:
# Set the 'date' column to a datetime format
avg_2['date'] = pd.to_datetime(avg_2['date'], format='%Y-%m-%d')

# Create a new dataframe with monthly dates from '2000-01-31' to '2021-05-31'
date_range = pd.date_range('2010-01-31', '2021-05-31', freq='M')
new_df = pd.DataFrame({'date': date_range})

# Use pivot to reshape the 'avg_2' dataframe
pivot_df = avg_2.pivot(index='date', columns='sa4', values='l01')

# Merge the 'new_df' and 'pivot_df' dataframes
merged_df = new_df.merge(pivot_df, on='date', how='outer')

# Sort the columns alphabetically by column name
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

# Fill missing values with NaN
avg_2_pivot = merged_df.fillna(np.nan).drop('date', axis=1)

In [126]:
# Set the 'date' column to a datetime format
sum_2['date'] = pd.to_datetime(sum_2['date'], format='%Y-%m-%d')

# Create a new dataframe with monthly dates from '2000-01-31' to '2021-05-31'
date_range = pd.date_range('2010-01-31', '2021-05-31', freq='M')
new_df = pd.DataFrame({'date': date_range})

# Use pivot to reshape the 'sum_2' dataframe
pivot_df = sum_2.pivot(index='date', columns='sa4', values='l01_s')

# Merge the 'new_df' and 'pivot_df' dataframes
merged_df = new_df.merge(pivot_df, on='date', how='outer')

# Sort the columns alphabetically by column name
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

# Fill missing values with NaN
sum_2_pivot = merged_df.fillna(np.nan).drop('date', axis=1)


In [127]:
# Set the 'date' column to a datetime format
avg_3['date'] = pd.to_datetime(avg_3['date'], format='%Y-%m-%d')

# Create a new dataframe with monthly dates from '2000-01-31' to '2021-05-31'
date_range = pd.date_range('2010-01-31', '2021-05-31', freq='M')
new_df = pd.DataFrame({'date': date_range})

# Use pivot to reshape the 'avg_3' dataframe
pivot_df = avg_3.pivot(index='date', columns='postcode', values='l012')

# Merge the 'new_df' and 'pivot_df' dataframes
merged_df = new_df.merge(pivot_df, on='date', how='outer')

# Sort the columns alphabetically by column name
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

# Fill missing values with NaN
avg_3_pivot = merged_df.fillna(np.nan).drop('date', axis=1)

In [128]:

df_yt = pd.concat([sum_1, sum_2_pivot, avg_3_pivot], axis=1)

# Convert the 'date' column to a datetime type
df_yt['date'] = pd.to_datetime(df_yt['date'])

# Set the 'date' column as the index
df_yt = df_yt.set_index('date')

In [129]:
df_yt

,l0_s,Capital Region,Central Coast,Central West,Coffs Harbour - Grafton,Far West and Orana,Hunter Valley exc Newcastle,Illawarra,Mid North Coast,Murray,...,2760,2763,2770,2774,2782,2795,2800,2829,2830,2871
date,,,,,,,,,,,,,,,,,,,,,
2010-01-31,1.288258,0.027196,0.023973,0.004776,0.130034,0.026090,0.045291,0.088781,0.000880,0.005368,...,0.010014,0.010993,0.010856,0.007293,0.001298,0.019524,-0.013081,0.010594,0.010197,-0.001667
2010-02-28,1.625062,-0.013162,0.062632,0.058484,0.021246,0.075471,0.077937,0.128832,0.064254,0.051052,...,0.000944,0.005322,-0.005627,0.008825,0.011302,0.021488,0.006007,0.051658,0.009192,0.030989
2010-03-31,1.722202,0.076356,-0.050916,0.034778,0.104249,0.018130,-0.056003,0.005223,0.054019,0.046827,...,0.016306,-0.006209,-0.005649,0.002476,0.002272,-0.017795,0.018227,0.000909,0.009155,0.034346
2010-04-30,1.700262,0.009337,0.067760,0.031473,-0.031351,0.036431,0.106638,0.089080,0.043788,0.119852,...,0.014515,0.017244,-0.001155,-0.001516,0.005592,0.012494,0.031635,0.066819,-0.001956,-0.012656
2010-05-31,1.347915,-0.045791,-0.017623,0.016356,0.050306,0.087570,0.093822,0.059250,-0.057030,0.001858,...,-0.006535,0.008089,-0.013910,-0.003970,0.000609,0.001372,-0.011512,0.045622,0.021829,0.026497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31,2.054224,0.119540,0.139027,0.067799,0.113528,0.093768,0.040779,0.087010,0.071603,0.057766,...,0.017009,0.016106,0.015315,0.012010,0.015709,0.013339,0.021675,0.075861,0.017907,0.032785
2021-02-28,5.250616,0.070001,0.233091,0.038266,0.200798,0.082995,0.164753,0.266743,0.183142,0.243050,...,0.055009,0.017706,0.043660,0.045412,0.041431,0.027885,0.033578,0.085019,0.023156,-0.023198
2021-03-31,6.956063,0.107420,0.369206,0.092421,0.125854,0.137303,0.238143,0.338204,0.135922,0.093295,...,0.057121,0.046268,0.042152,0.034592,0.059153,0.031921,0.057178,-0.000552,0.037045,0.003322


In [497]:
import numpy as np
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA

def base_errors(colname, M):
    # Get the data for the specified column
    data = df_yt[colname].values
    error_df = pd.DataFrame()

    # Initialize a matrix to store the forecasts
#     errors = np.zeros((N, data.shape[0] - M))

    # Loop over the rolling window
    for i in range(M, data.shape[0] ):
        N = data.shape[0] - M
        training_data = data[i-M:i]
        test_data = data[i:i+N]

        # Fit ARIMA model to training data
        model = auto_arima(training_data, d = None, seasonal=False, suppress_warnings=True, error_action="ignore", stepwise=True, trace=False)
        arima = ARIMA(training_data, order=model.order)
        arima_fit = arima.fit()
    
        # Make N-step-ahead forecast 
        forecast = arima_fit.forecast(steps=N)
        # Check if the forecast is longer than the test_data
        if len(forecast) > len(test_data):
        # Calculate the difference between the lengths of the two arrays
            difference = len(forecast) - len(test_data)
    
        # Add the extra values from the forecast array to the end of the test_data array
            test_data = np.append(test_data, forecast[-difference:])
        error_df['sim'+str(i)] = forecast - test_data
        # Calculate average MSE for each series
    return  error_df


In [234]:
colnames = df_yt.columns
M0  = 120
Dict_Errors = {}

for i, colname in enumerate(colnames):
    error_results = base_errors(colname,M0)
    Dict_Errors[colname] = error_results
    print(f'Progress: {i+1}/{len(colnames)}')


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Progress: 1/172
Progress: 2/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Progress: 3/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

Progress: 169/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressiv

Progress: 170/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/st

Progress: 171/172
Progress: 172/172
[2.35846691 0.06971073 0.10901014 0.04949835 0.08259238 0.1128299
 0.12865463 0.12098341 0.07079538 0.10433203 0.12309664 0.10412146
 0.11721863 0.07506842 0.07375872 0.08685234 0.03277628 0.16721307
 0.08788615 0.24168315 0.114947   0.27796408 0.17192264 0.04910723
 0.0935026  0.139549   0.02452723 0.05634227 0.07839557 0.03262146
 0.02703332 0.02574621 0.03111285 0.02283371 0.02710609 0.03206023
 0.02230314 0.03398687 0.02837789 0.0332253  0.02818875 0.02452705
 0.02371453 0.02576584 0.02469325 0.02935575 0.03259239 0.03235992
 0.02681769 0.02199358 0.03094127 0.03522643 0.03303118 0.03642742
 0.02715046 0.02851592 0.0311221  0.02989789 0.0312624  0.03162395
 0.02452723 0.0210902  0.02472869 0.03351794 0.02464774 0.02636291
 0.0216273  0.02902059 0.03095191 0.02870049 0.01955406 0.02420498
 0.01740144 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.       

In [498]:
base_errors('l0_s',M0)

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


,sim120,sim121,sim122,sim123,sim124,sim125,sim126,sim127,sim128,sim129,sim130,sim131,sim132,sim133,sim134,sim135,sim136
0,0.486039,-0.646892,1.134611,0.638867,1.344398,-0.136471,0.329419,-0.186115,-0.714207,-1.267230,-0.887271,-0.150116,0.529950,-3.385205,-1.928989,2.210179,-1.941356
1,-0.204383,0.544176,1.672685,2.095603,0.927955,0.210716,0.110860,-0.876889,-1.913524,-2.015173,-1.191041,0.371333,-2.831040,-5.294032,0.014140,0.896970,0.000000
2,0.947175,1.133396,3.038351,1.747379,1.056635,0.007893,-0.608937,-2.055477,-2.600918,-2.195450,-0.682364,-2.986041,-4.757314,-3.291676,-1.297762,0.000000,0.000000
3,1.500541,2.545393,2.607849,1.906683,0.687412,-0.697965,-1.813489,-2.724538,-2.727214,-1.577317,-3.958083,-4.900108,-2.801021,-4.499816,0.000000,0.000000,0.000000
4,2.880004,2.156855,2.692826,1.537160,-0.141619,-1.890172,-2.505773,-2.834618,-2.060955,-4.756020,-5.733868,-2.926206,-4.072583,0.000000,0.000000,0.000000,0.000000
5,2.461951,2.279838,2.256162,0.683392,-1.421954,-2.571522,-2.636626,-2.154016,-5.196754,-6.445814,-3.594463,-4.177465,0.000000,0.000000,0.000000,0.000000,0.000000
6,2.558160,1.877592,1.341752,-0.640049,-2.163748,-2.692689,-1.974603,-5.277130,-6.848300,-4.230192,-4.674738,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2.131631,0.994348,-0.036457,-1.437840,-2.324133,-2.022088,-5.114332,-6.917457,-4.598580,-5.242912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1.226366,-0.355644,-0.883706,-1.662322,-1.677062,-5.154220,-6.769518,-4.657814,-5.580903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,-0.143594,-1.177347,-1.152848,-1.083389,-4.821673,-6.802677,-4.523164,-5.631363,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [411]:
Dict_Errors['Capital Region']

,sim120,sim121,sim122,sim123,sim124,sim125,sim126,sim127,sim128,sim129,sim130,sim131,sim132,sim133,sim134,sim135,sim136
0,-0.023980,0.054081,-0.033933,0.033476,-0.029661,0.000745,0.003723,-0.041831,0.009109,-0.096659,-0.042410,-0.126290,-0.068731,-0.000586,-0.051995,-0.039375,0.014705
1,0.049116,-0.022566,0.026856,-0.022612,-0.006138,0.004241,-0.041540,-0.000466,-0.094614,-0.064409,-0.136963,-0.101967,-0.031490,-0.057896,-0.057937,-0.020298,0.000000
2,-0.023615,0.029344,-0.023689,-0.004370,0.002045,-0.041000,-0.000925,-0.097281,-0.063877,-0.142811,-0.104748,-0.055134,-0.091756,-0.066165,-0.039635,0.000000,0.000000
3,0.029101,-0.023045,-0.004321,0.002758,-0.042232,-0.000364,-0.097903,-0.065065,-0.142603,-0.107103,-0.055965,-0.093196,-0.085757,-0.049786,0.000000,0.000000,0.000000
4,-0.023123,-0.004061,0.003035,-0.041729,-0.001397,-0.097335,-0.065722,-0.143474,-0.106964,-0.057563,-0.093547,-0.087350,-0.061991,0.000000,0.000000,0.000000,0.000000
5,-0.004105,0.003215,-0.041406,-0.000937,-0.098327,-0.065152,-0.144139,-0.107767,-0.057439,-0.094982,-0.087583,-0.063620,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.003179,-0.041243,-0.000604,-0.097876,-0.066136,-0.143568,-0.108434,-0.058227,-0.094860,-0.088982,-0.063824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,-0.041277,-0.000444,-0.097541,-0.065686,-0.144550,-0.107862,-0.058895,-0.095646,-0.088862,-0.065216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,-0.000479,-0.097382,-0.065351,-0.144100,-0.108844,-0.058323,-0.096314,-0.089647,-0.065096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,-0.097416,-0.065191,-0.143765,-0.108395,-0.059305,-0.095742,-0.090315,-0.065880,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [236]:
# with open('Dict_Errors.pkl', 'wb') as f:
#     pickle.dump(Dict_Errors, f)

In [235]:

# for i, colname in enumerate(colnames):
#     error_results = base_errors(colname,M0,N0)
#     error_full = error_results.iloc[:, 0:-N0+1]
#     # Calculate the RMSE of each column in the error_full dataframe
#     rmse = np.sqrt(error_full.apply(lambda x: mean_squared_error(x, np.zeros(len(x))), axis=0))
#     # Calculate the average RMSE
#     avg_rmse_array[i] = np.mean(rmse)
#     errors_1[colname] = error_results.iloc[0, :]
#     print(f'Progress: {i+1}/{len(colnames)}')
# print(avg_rmse_array)

In [613]:
def forecast_base(M, h, Dict_Errors, df_yt):
    forecasts = pd.DataFrame({}, columns=df_yt.columns, index=df_yt.index)

    # Loop over all dataframes in Dict_Errors
    for key in Dict_Errors:
        # Find the row in Dict_Errors with index h-1 and extract all values before the last h column
        errors = Dict_Errors[key].iloc[h-1, :Dict_Errors[key].shape[1]-h+1]

        # Calculate the length of the extracted values from Dict_Errors
        length = len(errors)

        # Cut df_yt[key] into two parts
        first_part = df_yt[key].iloc[:-length]
        second_part = df_yt[key].iloc[-length:]

        # Add the second part of df_yt[key] and the extracted values from Dict_Errors to the forecast dataframe
        forecasts.loc[second_part.index, key] = second_part.values + errors.values


    return forecasts




In [614]:
# Extract the first row of each dataframe in Dict_Errors and combine them as new columns in a dataframe called error_1step
error_1step = pd.concat([Dict_Errors[key].iloc[0, :] for key in Dict_Errors], axis=1)

# Rename the columns of error_1step to match the keys in Dict_Errors
error_1step.columns = Dict_Errors.keys()

In [615]:
# Get a list of the column names of df_yt
sa4_lvls= list(avg_2_pivot.columns)
postcode_lvls = list(avg_3_pivot.columns)


In [616]:
import numpy as np
import pandas as pd

def generate_matrix_S(sa4_lvls, postcode_lvls, aveHPIdf_lv3):
    # Initialize a matrix of zeros with the same shape as the desired matrix S
    S = np.zeros((len(sa4_lvls), len(postcode_lvls)))

    # Loop over the rows of S
    for i in range(len(sa4_lvls)):
        # Get a boolean array indicating which rows of aveHPIdf_lv3 have sa4 equal to sa4_lvls[i]
        sa4_match = aveHPIdf_lv3['sa4'] == sa4_lvls[i]

        # Loop over the columns of S
        for j in range(len(postcode_lvls)):
            # Set S[i,j] to 1 if aveHPIdf_lv3['postcode'] is equal to postcode_lvls[j] and sa4_match is True
            if sa4_match.any() and (aveHPIdf_lv3['postcode'][sa4_match] == postcode_lvls[j]).any():
                S[i,j] = 1
    top_row = np.ones((1, len(postcode_lvls)))
    bottom_row = np.eye(len(postcode_lvls))
    Sfinal = np.vstack([top_row, S, bottom_row])

            

    # Convert the matrix to a dataframe with the appropriate row and column names
#     S = pd.DataFrame(S, index=sa4_lvls, columns=postcode_lvls)

    return Sfinal 


In [617]:
S = generate_matrix_S(sa4_lvls, postcode_lvls, aveHPIdf_lv3)

In [618]:

def mint_shrink(base_forecasts, base_error_1step):
    """
    Implements the MinT (Shrink) algorithm for forecast reconciliation.
    Given a set of N-step-ahead base forecasts and the in-sample one-step-ahead
    base forecast errors, returns the reconciled forecasts using the MinT (Shrink) method.
    """
    # Get the number of time series and the forecast horizon


    # Compute the empirical covariance matrix of the base forecast errors
    Sigma = np.cov(base_error_1step.T)

    correlation_matrix = base_error_1step.corr()

    # Calculate the variance of the correlation matrix
    correlation_matrix_var = correlation_matrix.apply(lambda x: (1 - x**2) / (len(base_error_1step) - 3)).values

    # Calculate the sum of the variances of the correlation estimates
    variance_sum = np.sum(correlation_matrix_var[np.triu_indices(len(base_error_1step), k=1)])

    # Calculate the sum of the squares of the correlation estimates
    correlation_sum = np.sum(correlation_matrix.values[np.triu_indices(len(base_error_1step), k=1)]**2)

    # Calculate the estimator for lambda_D
    lambda_D_hat = variance_sum / correlation_sum

    W_h = lambda_D_hat* D + (1 - lambda_D_hat) * Sigma

    m_star = S.shape[0] - S.shape[1]

    C = S[:len(sa4_lvls)+1,: ]

    U = np.hstack([np.identity(m_star), -C]).transpose()

    J = np.concatenate((np.zeros(( S.shape[1], m_star)), np.eye(S.shape[1])), axis=1)

    J_W_U = np.dot(np.dot(J, W_h), U)

    UWUinv = np.linalg.inv(np.dot(np.dot(U.transpose(), W_h),U) )

    J_W_UUWUinv =  np.dot(J_W_U,UWUinv )

    coefficient = np.dot(S, J - np.dot(J_W_UUWUinv ,U.transpose()))
    
    reconciled_forecasts =np.dot(coefficient,base_forecasts.values.transpose())
    return reconciled_forecasts



In [619]:
Dict_recon_Errors = {key: pd.DataFrame() for key in Dict_Errors}

for h in range(1, len(df_yt) - M0 + 1):
    mat = mint_shrink(forecast_base(M0, h, Dict_Errors, df_yt).dropna(), error_1step)
    i = 0
    for key in  Dict_recon_Errors.keys():
        forecasts = mat[i,:] 
        length = len(forecasts)
        testset = df_yt[key].iloc[-length:]
        new_row = (forecasts  -   testset).values
        new_row_length = len(new_row)
        if new_row_length < len(df_yt) - M0 :
            missing_cols = len(df_yt) - M0  - new_row_length
            new_row = np.append(new_row, np.full(missing_cols, np.nan))
        new_df = pd.DataFrame([new_row], columns=df_yt.index[-len(df_yt) + M0 :])

# concatenate the new dataframe to the original dataframe
        Dict_recon_Errors[key] = Dict_recon_Errors[key].append(new_df, ignore_index=True)
        i = i + 1
        
       
        
    


/var/folders/s1/vpd2prqd5fsc16qqsymrc9b00000gn/T/ipykernel_83240/3473400932.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Dict_recon_Errors[key] = Dict_recon_Errors[key].append(new_df, ignore_index=True)


In [671]:
# Dict_recon_Errors ['l0_s']

In [670]:
# Dict_Errors ['l0_s']

In [658]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def error_average(Dict_Name, h, Level):
    # check if level is valid
    if Level not in ['High', 'Mid', 'Low']:
        raise ValueError("Level must be 'High', 'Mid', or 'Low'.")

    # extract column names based on level
    if Level == 'Low':
        col_names = avg_3_pivot.columns
    elif Level == 'Mid':
        col_names = avg_2_pivot.columns
    else:
        col_names = ['l0_s']

    # extract dataframes from Dict_Name using column names as keys
    dfs = []
    for col in col_names:
        try:
            df = Dict_Name[col]
            dfs.append(df)
        except KeyError:
            print(f"Warning: Could not find dataframe for key {col}.")

    # calculate RMSE for each dataframe and find average by row
    rmse_by_row = []
    for df in dfs:
        df_error = df.iloc[0: h+1, 0:len(df_yt)-M-h].copy()
        # calculate RMSE by row
        rmse_by_row.append(np.sqrt(np.mean((df_error.values)**2)))

    # calculate average of average RMSE by row
    if rmse_by_row:
        average_rmse = np.mean([np.mean(rmse) for rmse in rmse_by_row])
    else:
        average_rmse = np.nan

    return average_rmse


In [689]:
error_average(Dict_Errors, 1, 'Low')

0.022902698117970154

In [690]:
error_average(Dict_recon_Errors, 1, 'Low')

0.020416014767204886

In [665]:
error_average(Dict_Errors, 3, 'Low')

0.02469242993098904

In [666]:
error_average(Dict_recon_Errors, 3, 'Low')

0.023136563534317752

In [688]:
error_average(Dict_Errors, 12, 'Low')

0.025182141192336114

In [687]:
error_average(Dict_recon_Errors, 12, 'Low')

0.025213541403191108

# Hierarchical Model

In [721]:
avg1 = avg_1.set_index("date")
avg2 = avg_2.set_index("date")
avg3 = avg_3.set_index("date")
df_dis =  pd.concat([avg1 , avg2.pivot(columns='sa4', values='l01'), avg3.pivot(columns='postcode', values='l012')],axis = 1)

In [737]:
def transform_df(df_dis):
    # create a copy of df_dis to avoid modifying the original dataframe
    df_transformed = df_dis.copy()
     # subtract the value of each column in group 3 by the value of the corresponding column in group 2
    for col in avg_3_pivot.columns:
        sa4 = aveHPIdf_lv3.loc[aveHPIdf_lv3['postcode'] == col]['sa4'].iloc[0]
        df_transformed[col] = df_transformed[col].sub(df_transformed[sa4], axis=0)
    # subtract the value under column 'l0' from columns in group 2
    df_transformed[avg_2_pivot.columns] = df_transformed[avg_2_pivot.columns].sub(df_transformed['l0'], axis=0)
    return df_transformed

In [741]:
 def reverse_transform_df(df_transformed):
    
    # create a copy of df_transformed to avoid modifying the original dataframe
    df_dis = df_transformed.copy()
    
    # add the value under column 'l0' back to columns in group 2
    df_dis[avg_2_pivot.columns] = df_dis[avg_2_pivot.columns].add(df_dis['l0'], axis=0)
    
    # add the value of each column in group 3 back to the value of the corresponding column in group 2
    for col in avg_3_pivot.columns:
        sa4 = aveHPIdf_lv3.loc[aveHPIdf_lv3['postcode'] == col]['sa4'].iloc[0]
        df_dis[col] = df_dis[col].add(df_dis[ sa4], axis=0)
    
    return df_dis


In [768]:
def base_errors_2(colname, M):
    if colname == 'l0' or colname in avg_2_pivot.columns: 
    # Get the data for the specified column
        data = df_l[colname].values
        error_df = pd.DataFrame()

    # Initialize a matrix to store the forecasts
#     errors = np.zeros((N, data.shape[0] - M))

    # Loop over the rolling window
        for i in range(M, data.shape[0] ):
            N = data.shape[0] - M
            training_data = data[i-M:i]
            test_data = data[i:i+N]

        # Fit ARIMA model to training data
            model = auto_arima(training_data, d = None, seasonal=False, suppress_warnings=True, error_action="ignore", stepwise=True, trace=False)
            arima = ARIMA(training_data, order=model.order)
            arima_fit = arima.fit()
    
        # Make N-step-ahead forecast 
            forecast = arima_fit.forecast(steps=N)
        # Check if the forecast is longer than the test_data
            if len(forecast) > len(test_data):
        # Calculate the difference between the lengths of the two arrays
                difference = len(forecast) - len(test_data)
    
        # Add the extra values from the forecast array to the end of the test_data array
                test_data = np.append(test_data, forecast[-difference:])
            error_df['sim'+str(i)] = forecast - test_data
        # Calculate average MSE for each series
        return  error_df
        
        

In [772]:
colnames = df_l.columns
M0  = 120
Dict_Errors_Hi = {}

for i, colname in enumerate(colnames):
    error_results = base_errors_2(colname,M0)
    Dict_Errors_Hi[colname] = error_results
    print(f'Progress: {i+1}/{len(colnames)}')


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Progress: 1/172
Progress: 2/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Use

Progress: 3/172
Progress: 4/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

Progress: 26/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Progress: 27/172
Progress: 28/172


/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/st

Progress: 29/172
Progress: 30/172
Progress: 31/172
Progress: 32/172
Progress: 33/172
Progress: 34/172
Progress: 35/172
Progress: 36/172
Progress: 37/172
Progress: 38/172
Progress: 39/172
Progress: 40/172
Progress: 41/172
Progress: 42/172
Progress: 43/172
Progress: 44/172
Progress: 45/172
Progress: 46/172
Progress: 47/172
Progress: 48/172
Progress: 49/172
Progress: 50/172
Progress: 51/172
Progress: 52/172
Progress: 53/172
Progress: 54/172
Progress: 55/172
Progress: 56/172
Progress: 57/172
Progress: 58/172
Progress: 59/172
Progress: 60/172
Progress: 61/172
Progress: 62/172
Progress: 63/172
Progress: 64/172
Progress: 65/172
Progress: 66/172
Progress: 67/172
Progress: 68/172
Progress: 69/172
Progress: 70/172
Progress: 71/172
Progress: 72/172
Progress: 73/172
Progress: 74/172
Progress: 75/172
Progress: 76/172
Progress: 77/172
Progress: 78/172
Progress: 79/172
Progress: 80/172
Progress: 81/172
Progress: 82/172
Progress: 83/172
Progress: 84/172
Progress: 85/172
Progress: 86/172
Progress: 87/1

In [785]:
# Create a list of keys to delete
keys_to_delete = []

# Iterate over the keys in Dict_Errors_Hi and identify any null dataframes
for key in Dict_Errors_Hi:
    if Dict_Errors_Hi[key] is None or Dict_Errors_Hi[key].empty:
        keys_to_delete.append(key)

# Delete the identified keys from Dict_Errors_Hi
for key in keys_to_delete:
    del Dict_Errors_Hi[key]

In [783]:
def forecast_base_2(M, h, Dict_Errors_Hi, df_l):
    forecasts = pd.DataFrame({}, columns=df_l.columns, index=df_l.index)

    # Loop over all dataframes in Dict_Errors
    for key in Dict_Errors_Hi:
        # Find the row in Dict_Errors with index h-1 and extract all values before the last h column
        errors = Dict_Errors_Hi[key].iloc[h-1, :Dict_Errors_Hi[key].shape[1]-h+1]

        # Calculate the length of the extracted values from Dict_Errors
        length = len(errors)

        # Cut df_yt[key] into two parts
        first_part = df_l[key].iloc[:-length]
        second_part = df_l[key].iloc[-length:]

        # Add the second part of df_yt[key] and the extracted values from Dict_Errors to the forecast dataframe
        forecasts.loc[second_part.index, key] = second_part.values + errors.values


    return forecasts


In [839]:
ARIMAforecasts = forecast_base_2(M0, 1, Dict_Errors_Hi, df_l)
ARIMAforecasts = ARIMAforecasts.reset_index(drop = True)


array([0.01823847755659106, -0.003221424562567205], dtype=object)

In [859]:
from sklearn.linear_model import LinearRegression
def ini_onestepforecast_hi(colname, M):
    if colname in avg_3_pivot.columns: 
    # Get the data for the specified column
        sa4 = aveHPIdf_lv3.loc[aveHPIdf_lv3['postcode'] == colname]['sa4'].iloc[0]
        results_array = np.empty((data.shape[0]-M,)) 
        for i in range(M, df_l.shape[0]):
            N = data.shape[0] - M
            train_start = i - M
            train_end = i - 1
            test_start = i
            test_end = i + N
            train_X = df_l[['l0', sa4]][train_start:train_end+1]
            train_Y = df_l[colname][train_start:train_end+1]
            model = LinearRegression().fit(train_X, train_Y)           
            test_X =np.array(ARIMAforecasts.iloc[test_start][['l0', sa4]]).reshape(1,-1)
            test_Y_pred = model.predict(test_X)
            results_array[i-M] 
        # Calculate average MSE for each series
        return  results_array
        

In [866]:
df_onestep = ARIMAforecasts.copy().dropna(how='all').reset_index(drop = True)
# Print the resulting DataFrame

In [868]:
for col in avg_3_pivot.columns:
    
    # Generate one-step forecasts using ini_onestepforecast_hi
    forecasts = ini_onestepforecast_hi(col, M0)
    
    # Add the forecasts to the ARIMAforecasts dataframe
    df_onestep[col] = forecasts

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

...

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

In [877]:
df_onestep_ori = reverse_transform_df(df_onestep)
df_onestep_ori = df_onestep_ori.filter(avg_3_pivot.columns, axis=1)
df_true = df_dis.copy().reset_index(drop = True)
df_true = df_true.tail(df_onestep_ori.shape[0]).reset_index(drop = True)
error_copula = (df_onestep_ori - df_true).dropna(axis = 1) 

17

In [897]:
import copulae
from copulae import EmpiricalCopula, pseudo_obs
from copulae.datasets import load_marginal_data
df  = error_copula

df_emp = df
u = pseudo_obs(df)
emp_cop = EmpiricalCopula(u, smoothing="none")
df_vol = EmpiricalCopula.to_marginals(emp_cop.random(2000, seed=10), df)

In [ ]:
def base_errors_2(colname, M):
    if colname == 'l0' or colname in avg_2_pivot.columns: 
    # Get the data for the specified column
        data = df_l[colname].values
        error_df = pd.DataFrame()

    # Initialize a matrix to store the forecasts
#     errors = np.zeros((N, data.shape[0] - M))

    # Loop over the rolling window
        for i in range(M, data.shape[0] ):
            N = data.shape[0] - M
            training_data = data[i-M:i]
            test_data = data[i:i+N]

        # Fit ARIMA model to training data
            model = auto_arima(training_data, d = None, seasonal=False, suppress_warnings=True, error_action="ignore", stepwise=True, trace=False)
            arima = ARIMA(training_data, order=model.order)
            arima_fit = arima.fit()
    
        # Make N-step-ahead forecast 
            forecast = arima_fit.forecast(steps=N)
        # Check if the forecast is longer than the test_data
            if len(forecast) > len(test_data):
        # Calculate the difference between the lengths of the two arrays
                difference = len(forecast) - len(test_data)
    
        # Add the extra values from the forecast array to the end of the test_data array
                test_data = np.append(test_data, forecast[-difference:])
            error_df['sim'+str(i)] = forecast - test_data
        # Calculate average MSE for each series
        return  error_df
 

def ini_onestepforecast_hi(colname, M):
    if colname in avg_3_pivot.columns: 
    # Get the data for the specified column
        sa4 = aveHPIdf_lv3.loc[aveHPIdf_lv3['postcode'] == colname]['sa4'].iloc[0]
        results_array = np.empty((data.shape[0]-M,)) 
        for i in range(M, df_l.shape[0]):
            N = data.shape[0] - M
            train_start = i - M
            train_end = i - 1
            test_start = i
            test_end = i + N
            train_X = df_l[['l0', sa4]][train_start:train_end+1]
            train_Y = df_l[colname][train_start:train_end+1]
            model = LinearRegression().fit(train_X, train_Y)           
            test_X =np.array(ARIMAforecasts.iloc[test_start][['l0', sa4]]).reshape(1,-1)
            test_Y_pred = model.predict(test_X)
            results_array[i-M] 
        # Calculate average MSE for each series
        return  results_array
        
        

In [977]:
def ARIMA_generator(M,i,colname):
    data = df_l[colname].values
    training_data_ARMA = data[i-M:i]
#     test_data_ARMA = data[i:i+N]
    model = auto_arima(training_data_ARMA, d = None, seasonal=False, suppress_warnings=True, error_action="ignore", stepwise=True, trace=False)
    arima = ARIMA(training_data_ARMA, order=model.order)
    arima_fit = arima.fit()
    return  arima,training_data_ARMA 

In [ ]:
S_u =S[:S.shape[0]- S.shape[1],:]
S_u = S_u / np.sum(S_u, axis=1, keepdims=True)

In [916]:
import random
Sim_Times = 100
def generate_hierforecast(M,i,simtimes = 100):
    if i < M:
        print("The training condition is not satisfied.")
        return None
    
    random_int = random.randint(0, 1999)
    
    ARIMAModel_Dict = {'l_0': {}}
    arima0, training_data_ARMA0  = ARIMA_generator(M, i, 'l0')
    ARIMAModel_Dict['Model'] = arima0
    ARIMAModel_Dict['Data'] = training_data_ARMA0
    for col in avg_2_pivot.columns:
        model_data = {}
        arima, training_data_ARIMA = ARIMA_generator(M, i, col)
        model_data['Model'] = arima
        model_data['Data'] = training_data_ARIMA
        ARIMAModel_Dict[col] = model_data


    for st in range(0,Sim_Times):
        arr_list = []
        for rolling in range (0, df_l.shape[0] - i):
            arr = []
            for postcode in list(avg_3_pivot.columns):
                sa4 = aveHPIdf_lv3.loc[aveHPIdf_lv3['postcode'] == colname]['sa4'].iloc[0]
                N = data.shape[0] - M
                train_start = i - M
                train_end = i - 1
                test_start = i
                train_X = df_l[['l0', sa4]][train_start:train_end+1]
                train_Y = df_l[colname][train_start:train_end+1]
                model = LinearRegression().fit(train_X, train_Y)           
                test_X =[ARIMAModel_Dict['l0']['Model'].fit().forecast(n=1)[0],ARIMAModel_Dict[sa4]['Model'].fit().forecast(n=1)].reshape(1,-1)
                test_Y_pred = model.predict(test_X) 
                l012_next = test_Y_pred + np.sum(test_X) + df_vol['postcode'].iloc[random_int]
                arr.append(l012_next)
                
            arr_list.append(arr)   


        

    
        
    
    

In [1011]:
random_int = random.randint(0, 1999)
arr = []
for postcode in list(avg_3_pivot.columns):
    sa4 = aveHPIdf_lv3.loc[aveHPIdf_lv3['postcode'] == colname]['sa4'].iloc[0]
    N = data.shape[0] - M
    train_start = i - M
    train_end = i - 1
    test_start = i
    train_X = df_l[['l0', sa4]][train_start:train_end+1]
    train_Y = df_l[colname][train_start:train_end+1]
    model = LinearRegression().fit(train_X, train_Y)           
    mytuple = ARIMAModel_Dict['l0']['Model'].fit().forecast(n=1),ARIMAModel_Dict[sa4]['Model'].fit().forecast(n=1)
    test_X = np.array(mytuple).reshape(1,-1)
    test_Y_pred = model.predict(test_X) 
    l012_next = test_Y_pred + np.sum(test_X) + df_vol[postcode].iloc[random_int]
    arr.append(l012_next[0])

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/pytho

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [1013]:
arr_newdata = np.dot(S_u, arr)

array([ 0.02242377,  0.00960389,  0.03309747,  0.02348957,  0.00516921,
        0.04820855,  0.02031732,  0.00168972,  0.02778931,  0.03112112,
        0.04617448,  0.02836423,  0.01029267,  0.03974832,  0.02810118,
        0.00182903,  0.02171202,  0.02011023,  0.03856399,  0.02471861,
        0.01422997,  0.02005255,  0.03663111,  0.02827339,  0.0182411 ,
        0.01563978, -0.0018281 ,  0.01581667,  0.01446354])

In [ ]:
def ARIMA_modifier(newdata):
    data = df_l[colnames].values
    training_data_ARMA = data[i-M:i]
#     test_data_ARMA = data[i:i+N]
    model = auto_arima(training_data, d = None, seasonal=False, suppress_warnings=True, error_action="ignore", stepwise=True, trace=False)
    arima = ARIMA(training_data, order=model.order)
    arima_fit = arima.fit()
    return  arima, training_data

In [978]:
i=150
ARIMAModel_Dict = {'l0': {}}
arima0, training_data_ARMA0  = ARIMA_generator(M0, i, 'l0')
ARIMAModel_Dict['l0']['Model'] = arima0
ARIMAModel_Dict['l0']['Data'] = training_data_ARMA0

for col in avg_2_pivot.columns:
    model_data = {}
    arima, training_data_ARIMA = ARIMA_generator(M0, i, col)
    model_data['Model'] = arima
    model_data['Data'] = training_data_ARIMA
    ARIMAModel_Dict[col] = model_data

/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/leiflyu/miniforge3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/leiflyu/minifo

In [985]:
ARIMAModel_Dict['l0']['Model'].fit().forecast(n=1)[0]

0.038016550447505855

(107, 172)